# Análisis Exploratorio

## Insurers

In [50]:
import requests
import numpy as np
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from pandas_profiling import ProfileReport
from funcs import *
import pingouin as pg


data = requests.get(
    "https://api.datamexico.org/tesseract/data.jsonrecords?cube=insurers&drilldowns=Ingress+Month%2CPatient+ID%2CHospital+Group%2CInterest+Supplies%2CMedical+Supply%2CState&measures=Treatment+Days%2CPatient+Treatment+Days%2CTreatment+Cost%2CSupply+Units%2CCost+per+Unit"
)
df = pd.DataFrame(data.json()["data"])
df["Ingress Month"] = pd.to_datetime(df["Ingress Month"] + "-01")
df['Ingress Year'] = df['Ingress Month'].dt.year
df = df[
    [
        "Ingress Month",
        "Patient ID",
        "Hospital Group",
        "Interest Supplies",
        "Medical Supply",
        "State",
        "Treatment Days",
        "Patient Treatment Days",
        "Treatment Cost",
        "Supply Units",
        "Cost per Unit",
    ]
]

covid_data = requests.get('https://api.datamexico.org/tesseract/data.jsonrecords?cube=gobmx_covid&drilldowns=Ingress+Month&measures=Cases')
covidDF = pd.DataFrame(covid_data.json()['data'])




Ya que los datos estan explotados por cada Medical Supply, se agrega por separado un dataframe de pacientes con su gasto total.

In [51]:
cols = ['Patient ID', 'Ingress Month', 'Interest Supplies', 'Hospital Group', 'Treatment Days', 'State']
patientDF = df.groupby(cols)
patientDF = patientDF['Treatment Cost'].sum()
patientDF = patientDF.reset_index()
patientDF = patientDF[patientDF['Treatment Cost'] > 0]

## Revisión de variación del costo de tratamiento en el tiempo por paciente

In [52]:
rDF = patientDF[patientDF['Interest Supplies'] == 'Insumo de Interes'].groupby('Ingress Month')['Treatment Cost'].mean()
rDF = rDF.reset_index()
gptDF = patientDF.groupby('Ingress Month')['Treatment Days'].mean().reset_index()

annotation1 = '<b>Media de Costo<br>de Tratamiento</br>'
annotation2 = '<b>Media de Días<br>de Tratamiento</b>'

fig = make_subplots(
    rows=2,
    specs=[[{"secondary_y": True}], [{"secondary_y": True}]]
    )

fig.add_trace(
        go.Scatter(
            x=rDF["Ingress Month"],
            y=rDF["Treatment Cost"],
            name='Mean Treatment Cost',
            marker=dict(size=13, color=CATEGORY_PALETTE[0]),
        ),
        row=1, col=1,
        secondary_y=True,
    )

fig.add_annotation(x='2021-07-01', y=rDF.iloc[-1, 1], text=annotation1, 
                   row=1, col=1, yref='y2', showarrow=False, yshift=25,
                   xshift=80, font=dict(color=CATEGORY_PALETTE[0], size=13))

fig.add_trace(
    go.Scatter(
        x=covidDF["Ingress Month"],
        y=covidDF["Cases"],
        name="Casos Confirmados de COVID",
        fill='tozeroy',
        fillcolor="rgba(0, 0, 0, 0.2)",
        marker_color="rgba(0, 0, 0, 0.2)",
        marker=dict(color="rgba(0, 0, 0, 0.2)"),
        line=dict(color="rgba(0, 0, 0, 0.2)"),
        line_shape="spline",
    ),
    secondary_y=False,
    row=1, col=1
)

fig.add_trace(
        go.Scatter(
            x=gptDF["Ingress Month"],
            y=gptDF["Treatment Days"],
            name='Mean Treatment Days',
            marker=dict(size=13, color=CATEGORY_PALETTE[2]),
        ),
        row=2, col=1,
        secondary_y=True,
    )

fig.add_annotation(x='2021-07-01', y=gptDF.iloc[-1, 1],
                   row=2, col=1, yshift=25, xshift=80,
                   text=annotation2, yref='y2', showarrow=False,
                   font=dict(color=CATEGORY_PALETTE[2], size=13))


fig.add_trace(
    go.Scatter(
        x=covidDF["Ingress Month"],
        y=covidDF["Cases"],
        name="Casos Confirmados de COVID",
        fill='tozeroy',
        fillcolor="rgba(0, 0, 0, 0.2)",
        marker_color="rgba(0, 0, 0, 0.2)",
        marker=dict(color="rgba(0, 0, 0, 0.2)"),
        line=dict(color="rgba(0, 0, 0, 0.2)"),
        line_shape="spline",
    ),
    secondary_y=False,
    row=2, col=1
)

fig.add_annotation(xref='paper', yref='paper', x=0, y=-0.065,
                   text='Fuente: Elaboración propia con datos de Insurers, DataMexico',
                   showarrow=False)

fig.update_layout(title_text="Figura 1. Comportamíento histórico", showlegend=False,
                  template="plotly_white",)
fig.update_xaxes(range=['2020-02-01', '2021-12-01'])
fig.update_yaxes(title_text="Casos confirmados de COVID", secondary_y=False)
fig.update_yaxes(title_text="", secondary_y=True)
fig.write_image("images/comportamiento_historico_Costo.png", height=800, width=1000)
fig.show()

## Revisión de Días de tratamiento por paciente

In [53]:
ptDF = df[['Patient ID', 'Treatment Days']].drop_duplicates()
fig = px.histogram(ptDF, x='Treatment Days')
fig.update_traces(marker_color=CATEGORY_PALETTE[0])
fig.update_layout(xaxis_title='Treatment Days', yaxis_title='Count', title='Treatment Days Histogram')
fig.show()

La mayoría de los casos entran dentro de los primeros 10 días de tratamiento. Llama la atención ver que el pico más grande del histograma sea en 0 días.
Se necesita apreciar si la cantidad de días de tratamiento cambió durante el tiempo.

La media parece ser estable durante todo un año, hasta que en Marzo del 2021 empieza a caer notablemente el promedio de días de tratamiento. 
Esto podría deberse a la entrada de las vacunas, lamentablemente el cubo Insurers termina los datos hasta Julio 2021 y no nos permite poder analizar esta hipótesis.

Vale la pena hacer un análisis de conteo de ceros durante el mismo tiempo.

In [54]:
zerosDF = patientDF[patientDF["Treatment Days"] == 0]
fig = px.histogram(
    zerosDF,
    x="Ingress Month",
    labels={"Ingress Month": "Mes de ingreso"},
    text_auto=True
)
fig.update_traces(marker_color=CATEGORY_PALETTE[0])
fig.update_layout(bargap=0.2, title='Conteo de Días de Tratamiento 0 en el tiempo', yaxis_title='', xaxis_title='')
fig.show()


Aunque hay un incremento notable de casos de Octubre 2020 hasta Marzo 2021, la variable vuelve a normalizarse. Haría falta más datos para poder comprbar si vuelven a aumentar y normalizarse en una cantidad más grande que la media de todos los zeros en el dataset.
Pese a esto, es claro que la cantidad de días se fue reduciendo en los útlmos 5 meses.

Con esto, un rango para bucketizar la variable de días de tratamiento aceptable es 5 días.

In [55]:
bins = np.array(range(0, 33, 4))
bins = np.append(bins, [np.Infinity])
labels = ['0-4', '5-8', '9-12', '13-16', '17-20', '21-24', '25-28', '29-32', '33-Inf']
df['TD Bucket'] = pd.cut(df['Treatment Days'], bins=bins, labels=labels)

In [56]:
ptDF = df[['Patient ID', 'TD Bucket']].drop_duplicates()
fig = px.histogram(ptDF, x='TD Bucket', text_auto=True)
fig.update_traces(marker_color=CATEGORY_PALETTE[0])
fig.update_xaxes(categoryorder='array', categoryarray=labels)
fig.update_layout(title='Conteo por Grupos de Días de Tratamiento', yaxis_title='', xaxis_title='')
fig.show()

Comportamiento de los artículos de interés contra no interés en el tiempo

In [57]:
patientDF2 = df.groupby(['Patient ID', 'Ingress Month', 'Interest Supplies'])
tpDF = patientDF2['Treatment Cost'].sum()
tpDF = tpDF.reset_index()
gDF = tpDF.groupby(['Ingress Month', 'Interest Supplies']).mean()
gDF = gDF.reset_index()

In [58]:
fig = px.line(gDF, x='Ingress Month', y='Treatment Cost', color='Interest Supplies')
fig.update_layout(legend=dict(orientation='h'), xaxis_title='', yaxis_title='Costo del Tratamiento')
fig.show()

La imagen arriba muestra el promedio de gastos general de insumos de interés y de no interés. Aunque las escalas para ambas categorías son distintas, se nota un comportamiento similar en cómo va decayendo los costos durante el tiempo, especialmente en los últimos 5 meses.
Ahora se hará un revisión del comportamiento de Costos de tratamiento por cada bucket de días de tratamiento.

In [59]:
patientDF[(patientDF['Ingress Month'].dt.year == 2020) & (patientDF['Interest Supplies'] == 'Insumo de Interes')]['Treatment Cost'].mean()

29741.465125877014

In [60]:
patientDF[(patientDF['Ingress Month'].dt.year == 2020)]['Treatment Cost'].mean()

329117.6258163657

In [61]:
tpDF = aggregate_patient(df, ['TD Bucket'])
gDF = mean_treatment(tpDF, ['TD Bucket'])

In [62]:
fig = px.line(gDF[gDF['Interest Supplies'] == 'Insumo de Interes'], x='Ingress Month', 
              y='Treatment Cost', color='Interest Supplies', facet_row='TD Bucket')
fig.update_layout(showlegend=False, xaxis_title='')
fig.update_yaxes(matches=None)
fig.for_each_annotation(lambda x: x.update(text=x.text.split('=')[-1]))
fig.write_image('images/buckets/facet_plot_interes.png', height=1200, width=1200)

fig = px.line(gDF[gDF['Interest Supplies'] == 'Insumo de no Interes'], x='Ingress Month', y='Treatment Cost', color='Interest Supplies', facet_row='TD Bucket')
fig.update_layout(showlegend=False, xaxis_title='')
fig.update_yaxes(matches=None)
fig.for_each_annotation(lambda x: x.update(text=x.text.split('=')[-1]))
fig.write_image('images/buckets/facet_plot_NOinteres.png', height=1200, width=1200)

![Promedio de Costos, por Buckets de Días de tratamiento](images/buckets/facet_plot_interes.png)

![](images/buckets/facet_plot_NOinteres.png)

Las imágenes de arriba muestra como se comporta la media de costo de tratamiento por cada bucket de días de tratamiento. Tiene sentido observar que mientras más días de tratamiento hay, más caro es el costo del tratamiento. Pero lo que es extraño es ver que los precios no 

In [63]:
for bucket in gDF["TD Bucket"].unique():
    iDF = gDF[gDF["TD Bucket"] == bucket]
    fig = make_subplots(rows=2, cols=1)
    isDF = iDF[iDF["Interest Supplies"] == "Insumo de Interes"]
    noDF = iDF[iDF["Interest Supplies"] == "Insumo de no Interes"]
    fig = fig.add_trace(
        go.Scatter(x=isDF["Ingress Month"], y=isDF["Treatment Cost"], name='Insumos de Interés'), 
        row=1, col=1,
        
    )
    fig = fig.add_trace(
        go.Scatter(x=noDF["Ingress Month"], y=noDF["Treatment Cost"], name='Insumos de no interés'), 
        row=2, col=1
    )
    fig.update_layout(
        height=800, width=1200, title_text=f"Mean Treatment Cost for Bucket {bucket}"
    )
    fig.write_image(f"./images/buckets/facet_plot_{bucket}.png", format="png", height=800, width=1200)


El código de arriba genera imágenes separadas para cada bucket de días de tratamiento. mostrando el promedio de costos de tratamiento a lo largo de la historia.

In [64]:
tpDF = aggregate_patient(df, ['TD Bucket', 'State'])
gDF = mean_treatment(tpDF, ['TD Bucket', 'State'])

El resultado muestra para el dataframe resultante que no hay información suficiente por estado para hacer una división por bucket de días. Es mejor sólo observar por estado sin buckets.

In [65]:
tpDF = aggregate_patient(df, ['State'])
gDF = mean_treatment(tpDF, ['State'])

In [66]:
for state in gDF["State"].unique():
    iDF = gDF[gDF["State"] == state]
    fig = make_subplots(rows=2, cols=1)
    isDF = iDF[iDF["Interest Supplies"] == "Insumo de Interes"]
    noDF = iDF[iDF["Interest Supplies"] == "Insumo de no Interes"]
    fig = fig.add_trace(
        go.Scatter(x=isDF["Ingress Month"], y=isDF["Treatment Cost"], name='Insumos de Interés'), 
        row=1, col=1,
        
    )
    fig = fig.add_trace(
        go.Scatter(x=noDF["Ingress Month"], y=noDF["Treatment Cost"], name='Insumos de no interés'), 
        row=2, col=1
    )
    fig.update_layout(
        height=800, width=1200, title_text=f"Mean Treatment Cost for {state}"
    )
    fig.write_image(f"./images/states/facet_plot_{state}.png", format="png", height=800, width=1200)

In [67]:
tpDF = aggregate_patient(df, ['Hospital Group'])
gDF = mean_treatment(tpDF, ['Hospital Group'])

In [68]:
fig = px.line(gDF, x='Ingress Month', y='Treatment Cost', color='Interest Supplies',
              facet_row='Hospital Group')
fig.show()

Las revisiones

In [69]:
tpDF = aggregate_patient(df, ['Hospital Group'])
tpDF.groupby(['Interest Supplies', 'Hospital Group'])['Treatment Cost'].mean()

Interest Supplies     Hospital Group
Insumo de Interes     High               29006.008630
                      Low                25855.293522
                      Medium             24616.004926
Insumo de no Interes  High              680720.184426
                      Low               511574.318390
                      Medium            626102.058252
Name: Treatment Cost, dtype: float64

In [70]:
tpDF.groupby(['Hospital Group', tpDF['Ingress Month'].dt.year])['Treatment Cost'].mean()

Hospital Group  Ingress Month
High            2020             312588.529412
                2021             441658.808568
Low             2020             269360.114035
                2021             295032.125399
Medium          2020             359093.982818
                2021             364141.474926
Name: Treatment Cost, dtype: float64

In [71]:
intDF = df[df['Interest Supplies'] == 'Insumo de Interes']
intDF = intDF[['Interest Supplies', 'Ingress Month', 'Treatment Cost', 'Medical Supply', 'Cost per Unit']].drop_duplicates()
pricesDF = intDF.groupby(['Ingress Month', 'Medical Supply'])
pricesDF = pricesDF['Cost per Unit'].mean()
pricesDF = pricesDF.reset_index()

In [72]:
patientDF[patientDF['Interest Supplies'] == 'Insumo de Interes'].groupby(patientDF['Ingress Month'].dt.year)['Treatment Cost'].mean()

Ingress Month
2020    29741.465126
2021    19934.638633
Name: Treatment Cost, dtype: float64

Los precios fueron variando durante el tiempo

## Pruebas de Hipótesis

### Diferencia de Medias de costos de Tratamientos por paciente 2020 vs 2021

Se revisa los promedios de costo de tratamiento por paciente entre 2020 y 2021. 
La primera prueba es si los promedios entre 2020 (Marzo-Julio) es diferente a 2021 (Marzo-Julio) para los insumos de interés.

$H_0$: El promedio de costo de tratamiento en 2020 es igual que en 2021

$H_1$: El promedio de costo de tratamiento en 2020 es diferente que en 2021

In [73]:
patientDF2020 = patientDF[
    (patientDF["Ingress Month"] >= "2020-03-01")
    & (patientDF["Ingress Month"] <= "2020-07-31")
    & (patientDF["Interest Supplies"] == "Insumo de Interes")
]
patientDF2021 = patientDF[
    (patientDF["Ingress Month"] >= "2021-03-01")
    & (patientDF["Interest Supplies"] == "Insumo de Interes")
]
newpDF = pd.concat([patientDF2020, patientDF2021])

newpDF.groupby(newpDF["Ingress Month"].dt.year)["Treatment Cost"].mean()


Ingress Month
2020    36548.506571
2021    19228.205128
Name: Treatment Cost, dtype: float64

In [74]:
newpDF.groupby(newpDF["Ingress Month"].dt.year)['Patient ID'].count()

Ingress Month
2020    837
2021    780
Name: Patient ID, dtype: int64

In [75]:
import matplotlib.pyplot as plt
from scipy.stats import t
# Prueba de hipotesis
# alpha = 0.05
# H0: El promedio de gasto en 2020 es igual a 2021
# H1: El promedio de gasto en 2020 es diferente a 2021

m2021 = patientDF2021['Treatment Cost'].mean()
m2020 = patientDF2020['Treatment Cost'].mean()
s2021 = patientDF2021['Treatment Cost'].std()
s2020 = patientDF2020['Treatment Cost'].std()
n2021 = len(patientDF2021)
n2020 = len(patientDF2020)

numerator = (m2021 - m2020)
denominator = np.sqrt((s2021**2/n2021)+(s2020**2/n2020))
t_stat = numerator/denominator
t_stat
degf = n2021 + n2020 - 2
p_value = t.cdf(t_stat, degf)
p_value # El resultado es estadísticamente significativo


# El resultado es estadísticamente significativo
pg.ttest(patientDF2020['Treatment Cost'], patientDF2021['Treatment Cost'], alternative='two-sided')

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,7.713118,1595.154452,two-sided,2.148301e-14,"[12915.73, 21724.87]",0.381418,2.144e+11,1.0


El resultado muestra que la diferencia entre promedio es estadísticamente significativo.

### Diferencia de Medias en Costo de tratamiento por Grupos Hospitalarios 

La siguiente prueba es para determinar si los promedios de costo de tratamiento son diferentes entre Grupos Hospitlarios para todo el conjunto de datos.

$H_0$: El promedio de costo de tratamiento es igual para todos los grupos hospitalarios

$H_1$: El promedio de costo de tratamiento es diferente para todos los grupos hospitalarios

In [76]:
results = pg.anova(patientDF[patientDF['Interest Supplies'] == 'Insumo de Interes'], dv = 'Treatment Cost', between='Hospital Group')
results

,Source,ddof1,ddof2,F,p-unc,np2
0,Hospital Group,2,3942,3.319554,0.03627,0.001681


In [77]:
pg.pairwise_tests(patientDF[patientDF['Interest Supplies'] == 'Insumo de Interes'], dv = 'Treatment Cost', between='Hospital Group', padjust='bonf')

,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,hedges
0,Hospital Group,High,Low,False,True,1.581718,1902.478586,two-sided,0.113880,0.341641,bonf,0.178,0.071981
1,Hospital Group,High,Medium,False,True,2.694685,1864.514775,two-sided,0.007109,0.021326,bonf,1.641,0.105128
2,Hospital Group,Low,Medium,False,True,0.706663,1797.432814,two-sided,0.479868,1.000000,bonf,0.056,0.028331


Los resultados muestran que las diferencias entre High-Medium son significativas

In [78]:
intDF = patientDF[patientDF['Interest Supplies'] == 'Insumo de Interes']
fig = go.Figure()
colors = {'Low': CATEGORY_PALETTE[0], 'Medium': CATEGORY_PALETTE[3], 'High': CATEGORY_PALETTE[2]}
for group in intDF['Hospital Group'].unique():
  iDF = intDF[intDF['Hospital Group'] == group]
  fig.add_trace(go.Box(y=iDF['Treatment Cost'], name=group, boxpoints='all', marker_color=colors[group]))
fig.update_yaxes(range=[0, 100000])
fig.show()

### Diferencia de Medias en Costo de tratamiento por Grupos Hospitalarios en 2020 y 2021 

La siguiente prueba es para determinar si los promedios de costo de tratamiento son diferentes entre Grupos Hospitlarios para 2020 y 2021, de Marzo a Julio en ambos años.

$H_0$: El promedio de costo de tratamiento es igual para todos los grupos hospitalarios

$H_1$: El promedio de costo de tratamiento es diferente para todos los grupos hospitalarios

In [79]:
# ANOVA de costos promedio por Nivel hospitalario (low, medium, high)
# Prueba de hipotesis
# H0: El promedio de gasto en 2020 es igual para todos los niveles
# H1: El promedio de gasto en 2020 es diferente para algún nivel
newpDF2020 = newpDF[(newpDF['Ingress Month'].dt.year == 2020)]
results = pg.anova(newpDF2020, dv = 'Treatment Cost', between='Hospital Group')
results

,Source,ddof1,ddof2,F,p-unc,np2
0,Hospital Group,2,834,2.330126,0.097917,0.005557


In [80]:
pg.pairwise_tests(newpDF2020, dv = 'Treatment Cost', between='Hospital Group', padjust='bonf')

,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,hedges
0,Hospital Group,High,Low,False,True,1.227352,413.327862,two-sided,0.220389,0.661166,bonf,0.226,0.118413
1,Hospital Group,High,Medium,False,True,2.013114,406.875092,two-sided,0.044761,0.134282,bonf,0.655,0.173642
2,Hospital Group,Low,Medium,False,True,0.594799,342.024917,two-sided,0.552371,1.000000,bonf,0.117,0.052993


Para 2020, ninguno de los grupos es estadísticamente significativo a un $\alpha$ de 5%

In [81]:
# ANOVA de costos promedio por Nivel hospitalario (low, medium, high)
# Prueba de hipotesis
# H0: El promedio de gasto en 2020 es igual para todos los niveles
# H1: El promedio de gasto en 2020 es diferente para algún nivel
newpDF2021 = newpDF[(newpDF['Ingress Month'].dt.year == 2021)]
results = pg.anova(newpDF2021, dv = 'Treatment Cost', between='Hospital Group')
results

,Source,ddof1,ddof2,F,p-unc,np2
0,Hospital Group,2,777,0.565313,0.568416,0.001453


In [82]:
pg.pairwise_tests(newpDF2021, dv = 'Treatment Cost', between='Hospital Group', padjust='bonf')

,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,hedges
0,Hospital Group,High,Low,False,True,1.351136,366.821087,two-sided,0.177485,0.532454,bonf,0.27,0.134720
1,Hospital Group,High,Medium,False,True,0.414572,490.065072,two-sided,0.678637,1.000000,bonf,0.113,0.032016
2,Hospital Group,Low,Medium,False,True,-0.834218,613.995846,two-sided,0.404482,1.000000,bonf,0.129,-0.062726


Para 2021, similar a 2020, ningún grupo muestra una diferencia estadísticamente significativa

### Diferencias de medias de los grupos hospitalarios 2020 vs 2021

In [83]:
mDF = newpDF.groupby(['Hospital Group', newpDF['Ingress Month'].dt.year])['Treatment Cost'].mean()
mDF

Hospital Group  Ingress Month
High            2020             42243.390558
                2021             21195.853659
Low             2020             36058.097826
                2021             17035.333333
Medium          2020             33604.047619
                2021             19769.680851
Name: Treatment Cost, dtype: float64

In [84]:
mDF = mDF.unstack()
mDF = mDF.reset_index()
mDF.columns = ['Hospital Group', '2020', '2021']
mDF['2020'] = mDF['2020'].apply(lambda x: np.round(x, 2))
mDF['2021'] = mDF['2021'].apply(lambda x: np.round(x, 2))


In [85]:
def compareMeans(vars, column):
  results = []
  for grupo in vars:
    df1 = newpDF[(newpDF[column] == grupo) & (newpDF['Ingress Month'].dt.year == 2020)]
    df2 = newpDF[(newpDF[column] == grupo) & (newpDF['Ingress Month'].dt.year == 2021)]
    results.append(pg.ttest(df1['Treatment Cost'], df2['Treatment Cost'], alternative='two-sided', paired=False))
  result = pd.concat(results, axis=0)
  result['grupo'] = vars
  result['alpha'] = 0.05
  result['relevant'] = result['p-val'] < result['alpha']
  return result

resultDF = compareMeans(['High', 'Medium', 'Low'], 'Hospital Group')
resultDF

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power,grupo,alpha,relevant
T-test,4.869536,369.365887,two-sided,0.000002,"[12548.16, 29546.92]",0.449657,7968.974,0.992672,High,0.05,True
T-test,4.067270,766.992315,two-sided,0.000052,"[7157.23, 20511.5]",0.290000,248.212,0.982988,Medium,0.05,True
T-test,4.726719,304.794943,two-sided,0.000003,"[11103.41, 26942.12]",0.486665,4246.21,0.998629,Low,0.05,True


In [86]:
fig = go.Figure()
colors = {'Low': CATEGORY_PALETTE[0], 'Medium': CATEGORY_PALETTE[3], 'High': CATEGORY_PALETTE[2]}
for i in mDF['Hospital Group']:
    dats = mDF[mDF['Hospital Group'] == i][['2020', '2021']]
    fig.add_trace(
        go.Scatter(
            x=[2020, 2021],
            y=[np.round(dats['2020'].values[0]), np.round(dats['2021'].values[0])],
            mode="lines+markers+text",
            text=[np.round(dats['2020'].values[0]), np.round(dats['2021'].values[0])],
            textposition=["middle left", "middle right"],
            marker=dict(size=12, color=colors[i]),
            name=i,
        )
    )

fig.add_annotation(yref='paper', x=2020, y=-0.05, text='2020', showarrow=False, font=dict(size=16))
fig.add_annotation(yref='paper', x=2021, y=-0.05, text='2021', showarrow=False, font=dict(size=16))
fig.add_shape(
    type="line",
    x0=2020,
    x1=2020,
    y0=0,
    y1=1,
    xref="x",
    yref="paper",
    layer="below",
)
fig.add_shape(
    type="line",
    x0=2021,
    x1=2021,
    y0=0,
    y1=1,
    xref="x",
    yref="paper",
    layer="below",
)
fig.update_xaxes(range=[2019.5, 2021.5], visible=False)
fig.update_layout(title_text="Cambio en el costo de tratamiento por Grupo Hospitalario (2020 vs 2021)", 
                  yaxis_title="Media de Gasto de bolsillo", xaxis_title="",
                  legend=dict(orientation="h", yanchor="bottom", y=1.002, xanchor="right", x=1),
                  template="plotly_white",)
fig.write_image("images/fig5.png", width=1000, height=800)
fig.show()


In [87]:
mDF['PCT_DIFF'] = mDF['2021']/mDF['2020']-1
mDF

,Hospital Group,2020,2021,PCT_DIFF
0,High,42243.39,21195.85,-0.498245
1,Low,36058.10,17035.33,-0.527559
2,Medium,33604.05,19769.68,-0.411688


Todos las diferencias entre años son estadísticamente significativas.

### Media de Costos de Tratamiento de 2020 a 2021 (Marzo-Julio en ambos años)

In [ ]:
newpDF.groupby(['Hospital Group'])['Treatment Cost'].mean()

Hospital Group
High      33548.690176
Low       25290.495283
Medium    27069.221106
Name: Treatment Cost, dtype: float64

### Media de Días de Tratamiento
En este caso se usa toda la base de Insurers

In [ ]:
df[['Patient ID', 'Treatment Days']].drop_duplicates()['Treatment Days'].mean()

11.996340150699677

### Media de Días de Tratamiento para 2020 y 2021

In [ ]:
patientDF.groupby([patientDF['Ingress Month'].dt.year])['Treatment Days'].mean()

Ingress Month
2020    12.242028
2021    11.707656
Name: Treatment Days, dtype: float64

### Media de Costos de Tratamiento de 2020 por Grupos Hospitalarios para Insumos de Interés

In [ ]:
patientDF[
    (patientDF["Ingress Month"].dt.year == 2020)
    & (patientDF["Treatment Days"] == 0)
    & (patientDF["Interest Supplies"] == "Insumo de Interes")
].groupby('Hospital Group')["Treatment Cost"].mean()


Hospital Group
High      61389.230769
Low       48780.224719
Medium    44358.625000
Name: Treatment Cost, dtype: float64

### Conteo de Casos con Días de Tratamiento 0 en 2020 para insumos de interés

In [ ]:
patientDF[
    (patientDF["Ingress Month"].dt.year == 2020)
    & (patientDF["Treatment Days"] == 0)
    & (patientDF["Interest Supplies"] == "Insumo de Interes")
]['Hospital Group'].value_counts()

Low       89
Medium    80
High      39
Name: Hospital Group, dtype: int64

### Media de Costos de Tratamiento en Casos D0 en 2020 para insumos de interés

In [ ]:
# ANOVA de costos promedio por Nivel hospitalario (low, medium, high) para pacientes con 0 días de tratamiento
# Prueba de hipotesis
# H0: El promedio de gasto en 2020 es igual para todos los niveles
# H1: El promedio de gasto en 2020 es diferente para algún nivel
intDF = patientDF[
    (patientDF["Ingress Month"].dt.year == 2020)
    & (patientDF["Treatment Days"] == 0)
    & (patientDF["Interest Supplies"] == "Insumo de Interes")
]
results = pg.anova(intDF, dv = 'Treatment Cost', between='Hospital Group')
results

,Source,ddof1,ddof2,F,p-unc,np2
0,Hospital Group,2,205,1.215306,0.298747,0.011718


In [ ]:
pg.pairwise_tests(intDF, dv = 'Treatment Cost', between='Hospital Group', padjust='bonf', alpha=0.1)

,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,hedges
0,Hospital Group,High,Low,False,True,1.112362,63.375491,two-sided,0.270184,0.810552,bonf,0.354,0.225858
1,Hospital Group,High,Medium,False,True,1.447804,70.707807,two-sided,0.152091,0.456274,bonf,0.524,0.288169
2,Hospital Group,Low,Medium,False,True,0.520663,161.112496,two-sided,0.603316,1.000000,bonf,0.189,0.080217


Ninguna comparación entre grupo es estadísticamente significativo.

### Conteo de casos con Días de Tratamiento 0 para insumos de interés

In [ ]:
results = patientDF[
    (patientDF["Treatment Days"] == 0)
    & (patientDF["Interest Supplies"] == "Insumo de Interes")
].groupby(["Hospital Group", patientDF["Ingress Month"].dt.year])["Patient ID"].count()
results = results.unstack()
results.reset_index(inplace=True)
results 

Ingress Month,Hospital Group,2020,2021
0,High,39,43
1,Low,89,61
2,Medium,80,76


In [89]:
fig = go.Figure(go.Scatter(x=[2020, 2021], y=results.iloc[0, 1:], mode='lines+markers+text', 
                           text=['High', ''], textposition=['middle left', 'middle right'],
                           marker=dict(size=12, color=CATEGORY_PALETTE[0]), 
                           name='High',))
fig.add_trace(go.Scatter(x=[2020, 2021], y=results.iloc[1, 1:], mode='lines+markers+text',
                         text=['Low', ''], textposition=['middle left', 'middle right'],
                         marker=dict(size=12, color=CATEGORY_PALETTE[1]), 
                         name='Low'))
fig.add_trace(go.Scatter(x=[2020, 2021], y=results.iloc[2, 1:], mode='lines+markers+text',
                         text=['Medium', ''], textposition=['middle left', 'middle right'],
                         marker=dict(size=12, color=CATEGORY_PALETTE[2]), 
                         name='Medium'))
fig.add_shape(type='line', x0=2020, x1=2020, y0=0, y1=1, xref='x', yref='paper', layer='below',)
fig.add_shape(type='line', x0=2021, x1=2021, y0=0, y1=1, xref='x', yref='paper', layer='below',)
fig.update_layout(title='Diferencias entre casos T0 de insumo de interes por nivel hospitalario')
fig.show()

IndexError: single positional indexer is out-of-bounds

In [ ]:
fig = px.histogram(patientDF[(patientDF['Ingress Month'].dt.year == 2020) & (patientDF['Treatment Days'] == 0) & (patientDF['Interest Supplies'] == 'Insumo de Interes')], x='Treatment Cost', nbins=50)
fig.update_layout(title='Histograma de costos de tratamiento para casos T0 de insumo de interes')
fig.show()

In [ ]:
supDF = df[
    ["Patient ID", "Cost per Unit", "Medical Supply", "Ingress Month", "Hospital Group"]
].drop_duplicates()
supDF = supDF[supDF["Medical Supply"] == "BARICITINIB"]


### 

In [99]:
def createFigure(df, supply):
    supDF = df[
        ["Patient ID", "Cost per Unit", "Medical Supply", "Ingress Month", "Hospital Group"]
    ].drop_duplicates()
    supDF = supDF[supDF["Medical Supply"] == supply]
    mincost1 = np.round(supDF['Cost per Unit'].min())
    maxcost1 = np.round(supDF['Cost per Unit'].max())
    supDF['labels'] = ''
    supDF.iloc[0, -1] = mincost1
    supDF.loc[df['Cost per Unit'] == supDF['Cost per Unit'].max(), 'labels'] = maxcost1
    
    figs = []
    symbols = {'Low': 'circle', 'Medium': 'square', 'High': 'triangle-up'}
    for i in supDF['Hospital Group'].unique():
        iDF = supDF[supDF['Hospital Group'] == i]
        figs.append(go.Scatter(x=iDF['Ingress Month'], y=iDF['Cost per Unit'], 
                               mode='markers', name=i, 
                               marker=dict(size=13, symbol= symbols[i], 
                                           color=CATEGORY_PALETTE[0],
                                           opacity=0.6)))
    return figs

fig = make_subplots(rows=1, cols=2)

fig.add_traces(data = createFigure(df, 'RUXOLITINIB'), rows=1, cols=2)
fig.add_traces(data = createFigure(df, 'BARICITINIB'), rows=1, cols=1)
fig.update_traces(textposition='top center', row=1, col=1)
fig.update_traces(textposition='top center', row=1, col=2)
fig.add_hrect(y0=22027.5, y1=42000, fillcolor='red', line_width=0, opacity=0.2, row=1, col=1)
fig.add_hline(y=22027.5, line_dash="dash", line_color="red", row=1, col=1)
fig.add_annotation(
    x="2020-03-01", y=24500, text="Costo máximo<br>permitido", textangle=0, showarrow=False,
    font=dict(size=16),
)


fig.add_hline(y=46250, line_dash='dash', line_color='blue', row=1, col=2)
fig.add_annotation(
    x="2020-03-01", y=51800, text="Costo máximo<br>permitido 1", textangle=0, showarrow=False,
    font=dict(size=16), row=1, col=2
)
fig.add_hrect(y0=46250, y1=92499, fillcolor='blue', line_width=0, opacity=0.2, row=1, col=2)
fig.add_hline(y=92500, line_dash='dash', line_color='red', row=1, col=2)
fig.add_annotation(
    x="2020-03-01", y=99000, text="Costo máximo<br>permitido 2", textangle=0, showarrow=False,
    font=dict(size=16), row=1, col=2
)
fig.add_hrect(y0=92500, y1=150000, fillcolor='red', line_width=0, opacity=0.2, row=1, col=2)

fig.update_layout(
    showlegend=False,
    title_text="Figura 5. Costos por unidad de BARICITINIB y RUXOLITINIB",
    xaxis_title="", 
)
fig.add_annotation(xref="paper", yref="paper", x=-0.01, y=-0.08, 
                   text="Fuente: Elaboración propia con datos de Insurers, DataMexico", 
                   showarrow=False)
fig.write_image("images/fig5.png", width=1200, height=700)
fig.show()


In [111]:
def createFigure(df, supply):
    supDF = df[
        ["Patient ID", "Cost per Unit", "Medical Supply", "Ingress Month", "Hospital Group"]
    ].drop_duplicates()
    supDF = supDF[supDF["Medical Supply"] == supply]
    mincost1 = np.round(supDF['Cost per Unit'].min())
    maxcost1 = np.round(supDF['Cost per Unit'].max())
    supDF['labels'] = ''
    supDF.iloc[0, -1] = mincost1
    supDF.loc[df['Cost per Unit'] == supDF['Cost per Unit'].max(), 'labels'] = maxcost1
    
    figs = []
    symbols = {'Low': 'circle', 'Medium': 'square', 'High': 'triangle-up'}
    for i in supDF['Hospital Group'].unique():
        iDF = supDF[supDF['Hospital Group'] == i]
        figs.append(go.Scatter(x=iDF['Ingress Month'], y=iDF['Cost per Unit'], 
                               mode='markers', name=i, 
                               marker=dict(size=13, symbol= symbols[i], 
                                           color=CATEGORY_PALETTE[0],
                                           opacity=0.6)))
    return figs

fig = make_subplots(rows=2, cols=1, subplot_titles=("BARICITINIB", "RUXOLITINIB"), vertical_spacing=0.1)

fig.add_traces(data = createFigure(df, 'RUXOLITINIB'), rows=2, cols=1)
fig.add_traces(data = createFigure(df, 'BARICITINIB'), rows=1, cols=1)
fig.update_traces(textposition='top center', row=1, col=1)
fig.update_traces(textposition='top center', row=2, col=1)
fig.add_hrect(y0=22027.5, y1=42000, fillcolor='red', line_width=0, opacity=0.2, row=1, col=1)
fig.add_hline(y=22027.5, line_dash="dash", line_color="red", row=1, col=1)
fig.add_annotation(
    x="2020-03-01", y=24500, text="Costo máximo<br>permitido", textangle=0, showarrow=False,
    font=dict(size=16),
)


fig.add_hline(y=46250, line_dash='dash', line_color='blue', row=2, col=1)
fig.add_annotation(
    x="2020-03-01", y=51800, text="Costo máximo<br>permitido 1", textangle=0, showarrow=False,
    font=dict(size=16), row=2, col=1
)
fig.add_hrect(y0=46250, y1=92499, fillcolor='blue', line_width=0, opacity=0.2, row=2, col=1)
fig.add_hline(y=92500, line_dash='dash', line_color='red', row=2, col=1)
fig.add_annotation(
    x="2020-03-01", y=99000, text="Costo máximo<br>permitido 2", textangle=0, showarrow=False,
    font=dict(size=16), row=2, col=1
)
fig.add_hrect(y0=92500, y1=150000, fillcolor='red', line_width=0, opacity=0.2, row=2, col=1)

fig.update_layout(
    showlegend=False,
    title_text="Figura 5. Costos por unidad de BARICITINIB y RUXOLITINIB",
    xaxis_title="", 
    template="plotly_white",
)
fig.update_yaxes(title_text="Costo por unidad", row=1, col=1)
fig.update_yaxes(title_text="Costo por unidad", row=2, col=1)
fig.add_annotation(xref="paper", yref="paper", x=-0.01, y=-0.04, 
                   text="Fuente: Elaboración propia con datos de Insurers, DataMexico", 
                   showarrow=False)
fig.write_image("images/fig5.png", height=1200, width=800)
fig.show()


In [ ]:
supDF = df[['Patient ID', 'Cost per Unit', 'Medical Supply', 'Ingress Month']].drop_duplicates()
supDF = supDF[supDF['Medical Supply'] == 'BARICITINIB'].groupby(['Ingress Month'])['Cost per Unit'].mean()
supDF = supDF.reset_index()

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
        go.Scatter(
            x=supDF["Ingress Month"],
            y=supDF["Cost per Unit"],
            name='Costo promedio de BARICITINIB por mes',
        ),
        secondary_y=False,
    )
fig.add_trace(
    go.Scatter(
        x=covidDF["Ingress Month"],
        y=covidDF["Cases"],
        name="COVID Curve",
        fill='tozeroy',
        fillcolor="rgba(0, 0, 0, 0.2)",
        marker_color="rgba(0, 0, 0, 0.2)",
        marker=dict(color="rgba(0, 0, 0, 0.2)"),
        line=dict(color="rgba(0, 0, 0, 0.2)"),
    ),
    secondary_y=True,
)
fig.show()

In [ ]:
supDF = df[['Patient ID', 'Cost per Unit', 'Medical Supply', 'Ingress Month']].drop_duplicates()
supDF = supDF[supDF['Medical Supply'] == 'RUXOLITINIB'].groupby(['Ingress Month'])['Cost per Unit'].mean()
supDF = supDF.reset_index()

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
        go.Scatter(
            x=supDF["Ingress Month"],
            y=supDF["Cost per Unit"],
            name='Costo promedio de RUXOLITINIB por mes',
        ),
        secondary_y=False,
    )
fig.add_trace(
    go.Scatter(
        x=covidDF["Ingress Month"],
        y=covidDF["Cases"],
        name="COVID Curve",
        fill='tozeroy',
        fillcolor="rgba(0, 0, 0, 0.2)",
        marker_color="rgba(0, 0, 0, 0.2)",
        marker=dict(color="rgba(0, 0, 0, 0.2)"),
        line=dict(color="rgba(0, 0, 0, 0.2)"),
    ),
    secondary_y=True,
)
fig.show()

## Proporción de Gupos Hospitalarios por Estado

In [91]:
hpDF = patientDF[['Hospital Group', 'State']]
hpDF = hpDF.groupby(['State']).value_counts().unstack().fillna(0)
hpDF['isMultiple'] = hpDF.apply(lambda row: np.sum([row.High > 0, row.Medium > 0, row.Low > 0]) > 1, axis=1)
hpDF['Total'] = hpDF.sum(axis=1)
hpDF['PCT'] = hpDF['Total'] / hpDF['Total'].sum()*100
hpDF = pd.concat([hpDF, 
           pd.DataFrame({'High': hpDF.High.sum(), 'Medium': hpDF.Medium.sum(), 'Low': hpDF.Low.sum()}, index=['Total'])])
hpDF = hpDF.sort_values(by='Total', ascending=True)

In [92]:
hpDF.style.format(precision=1, thousands=' ').background_gradient(cmap="YlGnBu", axis='columns')

,High,Low,Medium,isMultiple,Total,PCT
Nayarit,0.0,2.0,0.0,False,2.0,0.0
Colima,0.0,12.0,0.0,False,12.0,0.1
No Informado,0.0,0.0,16.0,False,16.0,0.2
San Luis Potosí,0.0,0.0,20.0,False,20.0,0.2
Aguascalientes,0.0,0.0,25.0,False,25.0,0.3
Michoacán de Ocampo,0.0,12.0,17.0,True,30.0,0.3
Guanajuato,2.0,12.0,18.0,True,33.0,0.4
Quintana Roo,0.0,38.0,14.0,True,53.0,0.6
Morelos,0.0,0.0,64.0,False,64.0,0.7
Tamaulipas,0.0,45.0,46.0,True,92.0,1.1


In [96]:
plotDF = hpDF.reset_index().iloc[0:-1, :]
fig = px.bar(plotDF, y='index', x='Total', orientation='h')
fig.update_layout(xaxis_title="Pacientes Tratados", yaxis_title="",
                  title=f'Figura 6. Estados que menor cantidad de pacientes trataron<br> tienen <span style="color: {CATEGORY_PALETTE[3]};"><b>sólo un tipo de grupo hospitalario</b></span>',
                  legend_yanchor="top", legend_y=0.1, legend_xanchor="left", legend_x=0.1,
                  legend_title_text='Grupo Hospitalario > 1', template='plotly_white')
fig.update_traces(marker_color=plotDF.isMultiple.map({True: CATEGORY_PALETTE[0], False: CATEGORY_PALETTE[3]}))
fig.add_annotation(xref="paper", yref="paper", x=-0.01, y=-0.09, showarrow=False,
                   text="Fuente: Elaboración propia con datos de Insurers, DataMexico",)
fig.write_image("images/fig4.png", width=1000, height=1000)
fig.show()

In [35]:
hpDF.isMultiple.value_counts(normalize=True)

True     0.625
False    0.375
Name: isMultiple, dtype: float64

In [36]:
hpDF[hpDF.isMultiple == False].sum()

High               0.0
Low              146.0
Medium           433.0
isMultiple           0
Total            579.0
PCT           6.729428
dtype: object

In [40]:
print(f'Low: {hpDF[hpDF.isMultiple == False].Low.sum()/579}')
print(f'Medium: {hpDF[hpDF.isMultiple == False].Medium.sum()/579}')
print(f'High: {hpDF[hpDF.isMultiple == False].High.sum()/579}')

Low: 0.25215889464594127
Medium: 0.7478411053540587
High: 0.0


In [42]:
hpDF[hpDF.isMultiple == False].Total.sum()/hpDF.Total.sum()

0.06729428172942817

Llama la atención ver que en el 37.5% de los estados sólo se tenga reportado un sólo tipo de grupo hospitalario. De estos, no existe ningún hospital High y la mayoría son Medium. ¿Podemos asumir que es un problema de acceso a la salud?